In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.

In [2]:
from module.db.db_model import DbModel
from module.code_matching.mintit import match_mintit_index
from module.code_matching.joongabi import match_joongabi_index
from module.code_matching.bunjang import match_bunjang_index
from module.code_matching.price_list import match_price_list
from module.code_matching.merge import merge_index

from sqlalchemy import Table, Column, Integer, String, Boolean, Date
from module.db.db_model import CreateTableIfNotExists

db_model = DbModel("main") #code_matching 테이블 없을경우 정의해줌

phone_list = db_model.get_table_dataframe("g5_phone_list")

mintit_index = match_mintit_index(db_model,phone_list)

joongabi_index = match_joongabi_index(db_model,phone_list)

bunjang_index = match_bunjang_index(db_model,phone_list)

price_list = match_price_list(db_model,phone_list)

result= merge_index(bunjang_index,joongabi_index,mintit_index,price_list,phone_list)

a = db_model.update_code_matching_orm(result)

db_model.end()





DB켜짐
DB꺼짐


In [5]:
a.apply(lambda d: d).drop_duplicates()

,original_code,storage,mintit_code,bunjang_code,joongabi_code,price_code
0,A1332,08,AIP4-08G,None,None,None
0,SM-N986,256,SM-N986,DGTLSSSMN986N256GB,SM-N986,SM-N986N
1,A1332,16,AIP4-16G,None,None,None
1,SM-N981,256,SM-N981,DGTLSSSMN981N256GB,SM-N981,SM-N981N
2,A1332,32,AIP4-32G,None,None,None
2,SM-N976,512,SM-N976-512G,DGTLSSGN10P5G512GB,SM-N976_512G,SM-N976N
3,A1387,08,AIP4S-08G,None,None,None
3,SM-N976,256,SM-N976-256G,DGTLSSGN10P5G256GB,SM-N976_256G,NaN
4,A1387,16,AIP4S-16G,None,None,None
4,SM-N971,256,SM-N971-256G,DGTLSSGN105G256GB,SM-N971_256G,SM-N971N
